# Create Dataset

In [15]:
import numpy as np

np.random.seed(42)
X_train = np.random.randint(0, 9, size=(100000, 2))
X_test = np.random.randint(0, 9, size=(1000, 2))

In [16]:
y_train = np.sum(X_train, axis=1)
y_test = np.sum(X_test, axis=1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [17]:
train_data_set = np.c_[X_train, y_train]
test_data_set = np.c_[X_test, y_test]

# construct dnn

In [18]:
import tensorflow as tf

In [24]:
n_inputs = 2
n_hidden1 = 5
n_hidden2 = 10
n_hidden3 = 50
n_output = 1
learning_rate = 0.005

In [25]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=(None, 2), name='X')
y = tf.placeholder(tf.float32, shape=(None), name='y')

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.leaky_relu, name='hidden1')
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.leaky_relu, name='hidden2')
    #hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.leaky_relu, name='hidden3')
    output = tf.layers.dense(hidden2, n_output, name='output')

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.square(output - y), name='loss')

with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope('eval'):
    accuracy = tf.reduce_mean(tf.square(output - y), name='acc')

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [26]:
def fetch_data(X, y, batch_size, epoch, batch_idx):
    n, m = X.shape
    np.random.seed(epoch + batch_idx)
    batch_ix = np.random.randint(n, size=batch_size)
    X_batch , y_batch = X[batch_ix], y[batch_ix]
    return X_batch, y_batch

## Train dnn

In [27]:
n_epochs = 40
batch_size = 200
n_batchies = int(np.ceil(X_train.shape[0] // 200))

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for batch_idx in range(n_batchies):
            X_batch, y_batch = fetch_data(X_train, y_train, batch_size, epoch, batch_idx)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 5 == 0:
            acc = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            print(epoch, 'acc: ', acc)
    final_acc = sess.run(accuracy, feed_dict={X: X_test, y: y_test})
    print('final_acc: ', final_acc)
    y_pred = output.eval(feed_dict={X: X_test})
    saver.save(sess, './my_models/add_dnn')

0 acc:  0.00039876677
5 acc:  7.649252e-06
10 acc:  1.7379119e-06
15 acc:  9.014135e-08
20 acc:  1.9947965e-05
25 acc:  2.2818489e-07
30 acc:  3.0819012e-07
35 acc:  1.2294107e-07
final_acc:  8.865162e-08


## Test

In [28]:

with tf.Session() as sess:
    saver.restore(sess, './my_models/add_dnn')
    y_pred = output.eval(session=sess, feed_dict={X: X_test})
    pred_set = np.c_[X_test, y_pred]
    print(pred_set)

INFO:tensorflow:Restoring parameters from ./my_models/add_dnn
[[ 2.          4.          6.00026178]
 [ 0.          7.          6.99981546]
 [ 4.          3.          7.00021267]
 ...
 [ 6.          3.          9.00031376]
 [ 0.          8.          7.999825  ]
 [ 6.          4.         10.00028324]]


いい感じ